# 줄리아를 생각하다(Think julia)

## Chapter 15. 구조체와 객체

**구조체(struct)** : 사용자가 정의하는 **복합 자료형(composite type)**

**예제1)** 카르테시안 좌표평면 상 점을 나타내는 자료형 Point 생성

In [1]:
struct Point
    x
    y
end

* **헤더:** 구조체 이름 Point 명시

* **본문:** 구조체의 속성(attribute), 다른 말로 필드(field) 정의

* **호출:** 구조체 Point를 마치 필드 값을 인수로 받는 함수로 취급하여 호출

* **생성자(constructor):** 구조체가 함수로 사용될 경우 구조체를 이르는 말


구조체는 객체를 생성하기 위한 공장과도 같음

* 새로운 객체를 만드는 것을 **인스턴스 생성** (instantiation)

* **인스턴스** (instance) : 자료형을 따라 생성된 객체

* 어떤 인스턴스를 출력하면, 줄리아는 인스턴스의 자료형과 속성값이 무엇인지를 보여 줌

In [2]:
p = Point(3.0, 4.0)

Point(3.0, 4.0)

### 15.2 구조체는 불변

* 필드 값을 얻으려면 도트(.) 표기법을 사용

* 표현식 p.x의 의미: 'p가 가리키는 객체로 가서 x의 값을 가져오시오'라는 뜻

* 구조체는 기본적으로 불변이라 생성한 이후에는 필드 값을 바꿀 수 없음

    - immutable struct of type Point cannot be changed

        + 장점 1) 실행효율이 좋음
        + 장점 2) 생성자에서 제공하는 불변식을 절대 위반할 수 없음
        + 장점 3) 불변 객체를 사용하는 코드가 가독성이 더 좋음

In [3]:
x = p.x
y = p.y

4.0

In [4]:
distance = sqrt(p.x^2 + p.y^2)

5.0

In [5]:
p.y = 1.0

LoadError: setfield!: immutable struct of type Point cannot be changed

### 15.3 가변 구조체

* 필요 시 ```mutable struct``` 이용 가변 복합 자료형 생성 가능

* 도트 표기법을 이용하면 가변 구조체 인스턴스의 값을 변경할 수 있음

In [6]:
mutable struct MPoint
    x
    y
end

blank = MPoint(0.0, 0.0)

MPoint(0.0, 0.0)

In [7]:
blank.x = 3.0 ; blank.y = 4.0 ;

println(blank)

MPoint(3.0, 4.0)


### 15.4 직사각형 예시

사각형의 한 꼭짓점과 폭, 높이를 지정하는 직사각형 자료형 생성

In [8]:
"""
Represents a rectangle.

fields: width, height, corner
"""
struct Rectangle
    width
    height
    corner
end

Rectangle

In [10]:
origin = MPoint(0.0, 0.0)

MPoint(0.0, 0.0)

In [11]:
box = Rectangle(100.0, 200.0, origin)

Rectangle(100.0, 200.0, MPoint(0.0, 0.0))

MPoint 객체가 Rectangle 객체 내에 내포(embedded)됨

### 15.5 인수로 쓰이는 인스턴스

* 인스턴스는 변수와 마찬가지 방식으로 함수의 인수로 전달할 수 있음

**예시1)** Point 객체를 인수로 받아서 수학에서 표기하는 방식으로 출력

In [12]:
function printpoint(p)
    println("($(p.x), $(p.y))")
end

printpoint (generic function with 1 method)

In [13]:
printpoint(blank)

(3.0, 4.0)


**예시2)** 인수로 전달된 가변 구조체 객체의 필드 수정

In [14]:
function movepoint!(p, dx, dy)
    p.x += dx
    p.y += dy
    nothing
end

movepoint! (generic function with 1 method)

In [15]:
origin = MPoint(0.0, 0.0)

MPoint(0.0, 0.0)

In [16]:
movepoint!(origin, 1.0, 2.0)

In [17]:
print(origin)

MPoint(1.0, 2.0)

In [19]:
# 불변 구조체인 Point의 객체를 넘기면 오류가 발생

movepoint!(p, 1.0, 2.0)

LoadError: setfield!: immutable struct of type Point cannot be changed

**예시3)** 불변 객체의 가변 특성값(필드) 수정

In [25]:
function moverectangle!(rect, dx, dy)
    movepoint!(rect.corner, dx, dy)
end

moverectangle! (generic function with 1 method)

In [21]:
box

Rectangle(100.0, 200.0, MPoint(0.0, 0.0))

In [26]:
moverectangle!(box, 1.0, 2.0)

In [27]:
box

Rectangle(100.0, 200.0, MPoint(1.0, 2.0))

Rectangle 구조체은 불변 객체이나,  Rectangle 구조체 origin 필드는 가변 구조체이므로 변경 가능 

_단! 불변 객체의 가변 속성에 재할당을 할 수는 없음!_

In [28]:
box.corner = MPoint(2.0, 3.0)

LoadError: setfield!: immutable struct of type Rectangle cannot be changed

### 15.6 반환값으로서의 인스턴스

함수는 인스턴스를 반환할 수 있음

**예시1)** Rectangle 객체를 인수로 받아 직사각형의 중심 좌표(Point 객체) 반환 함수 작성

In [29]:
function findcenter(rect)
    Point(rect.corner.x + rect.width/2, rect.corner.y + rect.height/2)
end

findcenter (generic function with 1 method)

In [30]:
center = findcenter(box)

Point(51.0, 102.0)

### 15.7 복사

* **깊은 복사(deepcopy)** : 객체에 내포된 내용까지 복사하는 방식. 별칭 생성을 shallow copy로 대비해서 사용하기도 함

In [31]:
p1 = MPoint(3.0, 4.0)
p2 = deepcopy(p1)

MPoint(3.0, 4.0)

In [32]:
p1 ≡ p2

false

In [33]:
p1 == p2

false

* 가변 객체의 경우 == 연산자의 동작은 === 연산자의 동작과 동일함

* 객체의 동일성(identical)만 확인하고 동등성(equivalent)는 확인하지 않음 -> 가변 복합 자료형의 경우 동등성을 판단할 수 있는 기준이 적어도 아직은 없으므로

### 15.8 디버깅

**예시1)** 존재하지 않는 필드에 접근할 때 발생하는 오류

In [34]:
p = Point(3.0, 4.0)

p.z = 1.0 


LoadError: type Point has no field z

* 오류 발생 시 typeof 함수 이용 객체의 타입 확인

* isa 이용 타입 여부 확인

* fieldnames 함수 이용 객체의 속성 확인

* isdefined 함수 이용 특정 속성이 있는지 여부 확인

In [35]:
# 오류 발생 시 typeof 함수 이용 객체의 타입 확인
typeof(p)

Point

In [36]:
# isa 이용 타입 여부 확인
p isa Point

true

In [37]:
# fieldnames 함수 이용 객체의 속성 확인
fieldnames(Point)

(:x, :y)

In [38]:
# isdefined 함수 이용 특정 속성이 있는지 여부 확인
isdefined(p, :x)

true

* ```:```는 심벌을 나타냄. **객체의 속성**, 즉 **필드** 임을 나타내며 타 언어와 다르게 필드명을 심볼 타입으로 처리

* 타 언어의 방식과 같이 문자열로 객체의 속성(필드명)을 확인하려고 하면 에러가 발생함

In [39]:
isdefined(p, :y)

true

In [40]:
fieldnames(MPoint)

(:x, :y)

In [41]:
fieldnames(Rectangle)

(:width, :height, :corner)

In [43]:
isdefined(p, "y")

LoadError: TypeError: in isdefined, expected Symbol, got a value of type String

### 15.9 용어집

* **구조체** (struct)

    - 이름 있는 필드의 모음을 갖고 있는 사용자 정의 자료형. 복합 자료형이라고도 함
```
```
* **속성** (attribute)

    - 객체와 관련된 이름 있는 값. 필드라고도 함
```
```
* **생성자** (constructor)

    - 어떤 자료형과 이름이 같고, 그 자료형의 인스턴스를 생성하는 기능을 하는 함수
```
```
* **인스턴스** (instunce)

    - 특정 자료형에 속하는 객체
```
```
* **인스턴스 생성** (instantiate)

    - 새로운 객체(인스턴스)를 만드는 행위
```
```
* **객체 도식** (object diagram)

    - 객체와 필드, 필드의 값을 표시한 도식
```
```
* **내포된 객체** (embedded object)

    - 다른 객체의 필드로 저장되어 있는 객체
```
```
* **깊은 복사** (deep copy)

    - 어떤 객체를 내포된 객체 모두를 포함해 복사하는 것. deepcopy 함수로 구현되어 있음